In [1]:
import mne
import matplotlib.pyplot as plt
import numpy as np
from datasets import concatenate_datasets,load_dataset

c:\Users\prans\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
new_df = load_dataset('pscotti/naturalscenesdataset')

To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)

In [2]:
%pip install mne

   ---------------------------------------- 0.0/7.4 MB ? eta -:--:--
   - -------------------------------------- 0.3/7.4 MB ? eta -:--:--
   ---- ----------------------------------- 0.8/7.4 MB 2.1 MB/s eta 0:00:04
   ------- -------------------------------- 1.3/7.4 MB 2.2 MB/s eta 0:00:03
   --------- ------------------------------ 1.8/7.4 MB 2.2 MB/s eta 0:00:03
   ------------ --------------------------- 2.4/7.4 MB 2.3 MB/s eta 0:00:03
   --------------- ------------------------ 2.9/7.4 MB 2.3 MB/s eta 0:00:02
   --------------- ------------------------ 2.9/7.4 MB 2.3 MB/s eta 0:00:02
   --------------- ------------------------ 2.9/7.4 MB 2.3 MB/s eta 0:00:02
   --------------- ------------------------ 2.9/7.4 MB 2.3 MB/s eta 0:00:02
   --------------- ------------------------ 2.9/7.4 MB 2.3 MB/s eta 0:00:02
   --------------- ------------------------ 2.9/7.4 MB 2.3 MB/s eta 0:00:02
   --------------- ------------------------ 2.9/7.4 MB 2.3 MB/s eta 0:00:02
   ----------------- -----


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
ds = load_dataset("Alljoined/05_125")

KeyboardInterrupt: 

In [ ]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['EEG', 'subject_id', 'session', 'block', 'trial', '73k_id', 'coco_id', 'curr_time'],
        num_rows: 34354
    })
    test: Dataset({
        features: ['EEG', 'subject_id', 'session', 'block', 'trial', '73k_id', 'coco_id', 'curr_time'],
        num_rows: 8716
    })
})


In [ ]:
#Combining Dataset
dataset = concatenate_datasets([ds['train'], ds['test']])
print(dataset)

Dataset({
    features: ['EEG', 'subject_id', 'session', 'block', 'trial', '73k_id', 'coco_id', 'curr_time'],
    num_rows: 43070
})


In [ ]:
eeg_sample = dataset[8]['EEG']
if isinstance(eeg_sample, list) or isinstance(eeg_sample, np.ndarray):
    print(np.array(eeg_sample).shape)


(64, 334)


In [5]:
# Visualize EEG Data (If It’s a Time Series)
eeg_sample = dataset[4]["EEG"]
plt.plot(eeg_sample)
plt.xlabel("Time")
plt.ylabel("EEG Signal")
plt.title("EEG Signal Sample")
plt.show()


NameError: name 'dataset' is not defined

In [ ]:
import numpy as np
from scipy.signal import butter, filtfilt

# Define the Butterworth bandpass filter function
def bandpass_filter(data, lowcut=1.0, highcut=50.0, fs=250, order=4):
    """
    Apply a Butterworth bandpass filter to the input EEG data.

    Parameters:
    - data: numpy array (1D or 2D) signal.
    - lowcut: lower frequency cutoff in Hz.
    - highcut: higher frequency cutoff in Hz.
    - fs: sampling frequency in Hz.
    - order: order of the filter.

    Returns:
    - filtered_data: filtered signal with same shape as input.
    """
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist

    # Create Butterworth bandpass filter coefficients
    b, a = butter(order, [low, high], btype='band')

    # Apply zero-phase filtering
    filtered_data = filtfilt(b, a, data, axis=0)
    return filtered_data

# Function to apply the filter to each record's "EEG" field
def apply_filter_to_record(example):
    """
    Apply the bandpass filter to the "EEG" field of a record while preserving other fields.

    Parameters:
    - example: A dictionary containing the record (e.g., EEG, subject_id, session, etc.).

    Returns:
    - example: The modified dictionary with filtered EEG data and all other fields preserved.
    """
    # Convert the EEG field to a NumPy array
    eeg_array = np.array(example['EEG'])

    # Apply the bandpass filter
    filtered_eeg = bandpass_filter(eeg_array, lowcut=1.0, highcut=50.0, fs=250, order=4)

    # Update the EEG field with the filtered data (convert back to list if needed)
    example['EEG'] = filtered_eeg.tolist()

    # Return the modified record with all fields preserved
    return example

filtered_dataset = dataset.map(apply_filter_to_record)


Parameter 'function'=<function apply_filter_to_record at 0x169d913f0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
Map: 100%|██████████| 43070/43070 [08:39<00:00, 82.96 examples/s]


In [ ]:
# Optionally, you can check a record to see the change
print(filtered_dataset[0]["subject_id"])

6


In [16]:
import os
import json

# Define the save directories
eeg_save_dir = "../data/EEG"
metadata_save_dir = "../data/metadata"

# Create directories if they don't exist
os.makedirs(eeg_save_dir, exist_ok=True)
os.makedirs(metadata_save_dir, exist_ok=True)

# Save each record as separate files in their respective folders
for i, record in enumerate(filtered_dataset):
    # Save the filtered EEG data in the EEG folder
    np.save(os.path.join(eeg_save_dir, f"eegSample_{i}.npy"), np.array(record['EEG']))

    # Save metadata (e.g., subject_id, session, etc.) in the metadata folder
    metadata = {key: value for key, value in record.items() if key != 'EEG'}
    with open(os.path.join(metadata_save_dir, f"metaData_{i}.json"), "w") as f:
        json.dump(metadata, f)

print(f"Filtered EEG data saved to {eeg_save_dir}")
print(f"Metadata saved to {metadata_save_dir}")

Filtered EEG data saved to ../data/EEG
Metadata saved to ../data/metadata


In [14]:
# load the json

import json
import os

# Load the metadata from the saved JSON files
metadata_dir = "../data/metadata"
metadata_files = sorted(os.listdir(metadata_dir))

metadata = []

for file in metadata_files:
    with open(os.path.join(metadata_dir, file), "r") as f:
        metadata.append(json.load(f))

# Display the metadata for the first record
print(metadata)

[{'subject_id': 6, 'session': 2, 'block': 1, 'trial': 1, '73k_id': 8509, 'coco_id': 541472, 'curr_time': 2.3359375}, {'subject_id': 6, 'session': 2, 'block': 1, 'trial': 2, '73k_id': 7007, 'coco_id': 536786, 'curr_time': 3.111328125}, {'subject_id': 6, 'session': 2, 'block': 1, 'trial': 12, '73k_id': 9722, 'coco_id': 544926, 'curr_time': 12.755859375}, {'subject_id': 6, 'session': 2, 'block': 1, 'trial': 13, '73k_id': 3157, 'coco_id': 584, 'curr_time': 13.5546875}, {'subject_id': 6, 'session': 2, 'block': 1, 'trial': 14, '73k_id': 4835, 'coco_id': 5684, 'curr_time': 14.369140625}, {'subject_id': 6, 'session': 2, 'block': 1, 'trial': 15, '73k_id': 3687, 'coco_id': 2270, 'curr_time': 15.138671875}, {'subject_id': 6, 'session': 2, 'block': 1, 'trial': 16, '73k_id': 5542, 'coco_id': 7932, 'curr_time': 16.71484375}, {'subject_id': 6, 'session': 2, 'block': 1, 'trial': 3, '73k_id': 6524, 'coco_id': 273147, 'curr_time': 3.9375}, {'subject_id': 6, 'session': 2, 'block': 1, 'trial': 4, '73k_id'

In [16]:
# I want to fetch image from coco dataset based on coco_id
coco_ids = []
for i in metadata:
    coco_id = i['coco_id']
    coco_ids.append(coco_id)

print(coco_ids)



[541472, 536786, 544926, 584, 5684, 2270, 7932, 273147, 535871, 262145, 393907, 8053, 280484, 267699, 265150]


In [9]:
%pip install pycocotools

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
from pycocotools.coco import COCO
import requests
from PIL import Image
from io import BytesIO

# Path to the COCO annotations file
# Update this based on the dataset you use
annotation_file = 'instances_val2017.json'

# Initialize COCO API
coco = COCO(annotation_file)

# Replace with your desired COCO image ID
coco_id = 123456  # Example ID, replace with a real one

# Get image metadata
img_info = coco.loadImgs([coco_id])[0]
img_url = img_info['coco_url']

# Fetch and display the image
response = requests.get(img_url)
img = Image.open(BytesIO(response.content))
img.show()

loading annotations into memory...


FileNotFoundError: [Errno 2] No such file or directory: 'instances_val2017.json'

In [3]:
import numpy as np

# Load the .npy file
data = np.load("../data/EEG/eegSample_0.npy")

# Check the shape and content
print("Shape of the data:", data.shape)
print("Data preview:", data)

Shape of the data: (64, 334)
Data preview: [[-3.83077682e-06 -3.80320487e-06 -1.20690662e-06 ...  1.43596026e-06
   1.88553891e-06  3.09111222e-06]
 [-2.99848788e-06 -2.99592261e-06 -1.15138412e-06 ... -5.50029683e-06
  -2.18943145e-06  6.91825658e-07]
 [-2.78665095e-06 -1.79202282e-06  7.11380555e-07 ... -9.35139905e-06
  -1.81484233e-06  1.63937415e-06]
 ...
 [ 9.37695695e-07  1.25180888e-06 -5.15266395e-07 ...  2.61801563e-07
  -1.51854740e-06 -2.30763828e-06]
 [ 1.19188521e-06  1.10450084e-06 -3.98136013e-07 ... -2.91526414e-07
  -1.65842973e-06 -2.42699743e-06]
 [ 8.62502145e-07  3.10742059e-07 -5.40157067e-07 ...  2.57219228e-07
  -1.09397924e-06 -1.54384293e-06]]


In [4]:
len(data)

64